In [50]:
import pandas as pd
from gurobipy import Model, GRB, quicksum

import matplotlib.pyplot as plt

In [51]:
WORKING_DIR = "/Users/junyangwu/Junyang Wu/Study/Columbia/2025.9-2025.12/4004 Optimization model and methods/Group Study/Project 2"

In [52]:
df = pd.read_csv(f'{WORKING_DIR}/games.csv')
df

,Date,Visitor,PTS,Home,PTS.1,Attend.,LOG,Arena,Notes
0,"Sat, Nov 01, 2025",Golden State Warriors,NaN,Boston Celtics,NaN,"19,000",7:30 PM,TD Garden,NaN
1,"Sat, Nov 01, 2025",Los Angeles Lakers,NaN,New York Knicks,NaN,"19,400",7:30 PM,Madison Square Garden,NaN
2,"Sat, Nov 01, 2025",Denver Nuggets,NaN,Brooklyn Nets,NaN,"17,500",7:30 PM,Barclays Center,NaN
3,"Sat, Nov 01, 2025",Phoenix Suns,NaN,Philadelphia 76ers,NaN,"19,650",7:30 PM,Wells Fargo Center,NaN
4,"Sat, Nov 01, 2025",Houston Rockets,NaN,Toronto Raptors,NaN,"19,600",7:30 PM,Scotiabank Arena,NaN
...,...,...,...,...,...,...,...,...,...
123,"Thu, Dec 25, 2025",Philadelphia 76ers,NaN,Phoenix Suns,NaN,"17,450",7:30 PM,Footprint Center,Christmas Day
124,"Thu, Dec 25, 2025",Toronto Raptors,NaN,Houston Rockets,NaN,"17,800",7:30 PM,Toyota Center,Christmas Day
125,"Thu, Dec 25, 2025",Miami Heat,NaN,Dallas Mavericks,NaN,"19,800",7:30 PM,American Airlines Center,Christmas Day
126,"Thu, Dec 25, 2025",Chicago Bulls,NaN,Atlanta Hawks,NaN,"17,250",7:30 PM,State Farm Arena,Christmas Day


In [ ]:
df = df.rename(columns={
    "Date": "date",
    "Visitor": "visitor",
    "Home": "home",
    "Arena": "arena"
})

df = df[["date", "visitor", "home", "arena"]].copy()

df["date"] = pd.to_datetime(df["date"], errors="coerce")
df["date_str"] = df["date"].dt.strftime("%Y-%m-%d")

teams = sorted(pd.unique(pd.concat([df["visitor"], df["home"]], ignore_index=True)))
dates = sorted(df["date_str"].unique().tolist())

print("Teams:", teams)
print("Total teams:", len(teams))
print("Total dates:", len(dates))

Teams: ['Atlanta Hawks', 'Boston Celtics', 'Brooklyn Nets', 'Chicago Bulls', 'Cleveland Cavaliers', 'Dallas Mavericks', 'Denver Nuggets', 'Golden State Warriors', 'Houston Rockets', 'Los Angeles Lakers', 'Miami Heat', 'Milwaukee Bucks', 'New York Knicks', 'Philadelphia 76ers', 'Phoenix Suns', 'Toronto Raptors']
Total teams: 16
Total dates: 16


In [ ]:
home_dates = (
    df.groupby("home")["date_str"]
      .agg(list)
      .to_dict()
)
for t in teams:
    home_dates.setdefault(t, [])

away_dates = (
    df.groupby("visitor")["date_str"]
      .agg(list)
      .to_dict()
)
for t in teams:
    away_dates.setdefault(t, [])

print("\nExample home_dates['Boston Celtics']:", home_dates.get("Boston Celtics", []))
print("Example away_dates['Boston Celtics']:", away_dates.get("Boston Celtics", []))


Example home_dates['Boston Celtics']: ['2025-11-01', '2025-11-07', '2025-11-13', '2025-11-19', '2025-11-23', '2025-11-28']
Example away_dates['Boston Celtics']: ['2025-11-03', '2025-11-05', '2025-11-11', '2025-11-15', '2025-11-17', '2025-11-21', '2025-11-27', '2025-11-29', '2025-12-01', '2025-12-25']


In [ ]:
home_vs = pd.crosstab(index=df["home"], columns=df["visitor"]).reindex(
    index=teams, columns=teams, fill_value=0
)

away_vs = pd.crosstab(index=df["visitor"], columns=df["home"]).reindex(
    index=teams, columns=teams, fill_value=0
)

b = {(i, j): int(home_vs.loc[i, j]) for i in teams for j in teams}
c = {(i, j): int(away_vs.loc[i, j]) for i in teams for j in teams}

print("\nSample b_ij (home_vs counts) for 'Boston Celtics':")
print(home_vs.loc["Boston Celtics"])


Sample b_ij (home_vs counts) for 'Boston Celtics':
visitor
Atlanta Hawks            0
Boston Celtics           0
Brooklyn Nets            0
Chicago Bulls            0
Cleveland Cavaliers      0
Dallas Mavericks         1
Denver Nuggets           0
Golden State Warriors    1
Houston Rockets          0
Los Angeles Lakers       0
Miami Heat               1
Milwaukee Bucks          1
New York Knicks          0
Philadelphia 76ers       1
Phoenix Suns             1
Toronto Raptors          0
Name: Boston Celtics, dtype: int64


In [ ]:

team_timezone = {
    "Boston Celtics": 0,          # TD Garden, Boston, MA (Eastern)
    "New York Knicks": 0,         # Madison Square Garden, NY (Eastern)
    "Brooklyn Nets": 0,           # Barclays Center, Brooklyn, NY (Eastern)
    "Philadelphia 76ers": 0,      # Wells Fargo Center, Philadelphia, PA (Eastern)
    "Toronto Raptors": 0,         # Scotiabank Arena, Toronto, ON (Eastern)
    "Miami Heat": 0,              # Kaseya Center, Miami, FL (Eastern)
    "Chicago Bulls": -1,          # United Center, Chicago, IL (Central)
    "Milwaukee Bucks": -1,        # Fiserv Forum, Milwaukee, WI (Central)
    "Los Angeles Lakers": -3,     # Crypto.com Arena, Los Angeles, CA (Pacific)
    "Golden State Warriors": -3,  # Chase Center, San Francisco, CA (Pacific)
    "Phoenix Suns": -2,           # Footprint Center, Phoenix, AZ (Mountain-ish, simplified as -2)
    "Houston Rockets": -1,        # Toyota Center, Houston, TX (Central)
    "Dallas Mavericks": -1,       # American Airlines Center, Dallas, TX (Central)
    "Atlanta Hawks": 0,           # State Farm Arena, Atlanta, GA (Eastern)
    "Cleveland Cavaliers": 0,     # Rocket Mortgage FieldHouse, Cleveland, OH (Eastern)
    "Denver Nuggets": -2          # Ball Arena, Denver, CO (Mountain)
}

for t in teams:
    if t not in team_timezone:
        raise ValueError(f"Missing timezone info for team: {t}")


In [ ]:
m = Model("NBA_schedule_with_timezone")
m.Params.OutputFlag = 1

x = {}
for i in teams:
    for d in home_dates[i]:
        for j in teams:
            if i == j:
                continue
            x[(i, j, d)] = m.addVar(vtype=GRB.BINARY,
                                    name=f"x[{i},{j},{d}]")

m.update()

for i in teams:
    for d in home_dates[i]:
        m.addConstr(
            quicksum(x[(i, j, d)] for j in teams if j != i) == 1,
            name=f"home_one[{i},{d}]"
        )

for i in teams:
    for d in away_dates[i]:
        m.addConstr(
            quicksum(
                x[(j, i, d)] for j in teams
                if j != i and (j, i, d) in x
            ) == 1,
            name=f"away_one[{i},{d}]"
        )

for i in teams:
    for j in teams:
        if i == j:
            continue
        m.addConstr(
            quicksum(
                x[(i, j, d)]
                for d in home_dates[i]
                if (i, j, d) in x
            ) == b[(i, j)],
            name=f"pair_home[{i},{j}]"
        )

for i in teams:
    for j in teams:
        if i == j:
            continue
        m.addConstr(
            quicksum(
                x[(j, i, d)]
                for d in away_dates[i]
                if (j, i, d) in x
            ) == c[(i, j)],
            name=f"pair_away[{i},{j}]"
        )

# def game_timezone_expr(i, d):
#     tz_i = team_timezone[i]
#     if d in home_dates[i]:
#         return tz_i

#     if d in away_dates[i]:
#         return quicksum(
#             team_timezone[j] * x[(j, i, d)]
#             for j in teams
#             if j != i and (j, i, d) in x
#         )

#     return 0

# team_play_dates = {}
# for i in teams:
#     dates_i = sorted(set(home_dates[i]) | set(away_dates[i]))
#     team_play_dates[i] = dates_i

# THRESHOLD = 3.0

# for i in teams:
#     dates_i = team_play_dates[i]
#     if len(dates_i) < 3:
#         continue

#     for k in range(len(dates_i) - 2):
#         d1, d2, d3 = dates_i[k], dates_i[k+1], dates_i[k+2]

#         Z1 = game_timezone_expr(i, d1)
#         Z2 = game_timezone_expr(i, d2)
#         Z3 = game_timezone_expr(i, d3)

#         d12_plus = m.addVar(lb=0.0, vtype=GRB.CONTINUOUS,
#                             name=f"d12_plus[{i},{k}]")
#         d12_minus = m.addVar(lb=0.0, vtype=GRB.CONTINUOUS,
#                             name=f"d12_minus[{i},{k}]")
#         d23_plus = m.addVar(lb=0.0, vtype=GRB.CONTINUOUS,
#                             name=f"d23_plus[{i},{k}]")
#         d23_minus = m.addVar(lb=0.0, vtype=GRB.CONTINUOUS,
#                             name=f"d23_minus[{i},{k}]")

#         # Z2 - Z1 = d12_plus - d12_minus
#         m.addConstr(Z2 - Z1 == d12_plus - d12_minus, name=f"tz_diff12_eq[{i},{k}]")
#         # Z3 - Z2 = d23_plus - d23_minus
#         m.addConstr(Z3 - Z2 == d23_plus - d23_minus, name=f"tz_diff23_eq[{i},{k}]")

#         # |Δ1| + |Δ2| = (d12_plus + d12_minus) + (d23_plus + d23_minus) <= THRESHOLD
#         m.addConstr(
#             d12_plus + d12_minus + d23_plus + d23_minus <= THRESHOLD,
#             name=f"tz_trip_bound[{i},{k}]"
#         )

m.update()
m.setObjective(0.0, GRB.MINIMIZE)
m.optimize()

Set parameter OutputFlag to value 1
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 24.1.0 24B2082)

CPU model: Apple M4 Pro
Thread count: 12 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 736 rows, 1920 columns and 5888 nonzeros
Model fingerprint: 0x038de529
Variable types: 0 continuous, 1920 integer (1920 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 428 rows and 1478 columns
Presolve time: 0.00s
Presolved: 308 rows, 442 columns, 1386 nonzeros
Variable types: 0 continuous, 442 integer (442 binary)
Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.01 work units)
Thread count was 12 (of 12 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.00000000000

In [ ]:

if m.status in [GRB.OPTIMAL, GRB.SUBOPTIMAL]:
    print("\n===== Found a feasible schedule (respecting timezone constraint) =====\n")
    schedule_rows = []

    for d in dates:
        # 找到该日期的所有主场比赛
        for i in teams:
            if d not in home_dates[i]:
                continue
            for j in teams:
                if i == j:
                    continue
                key = (i, j, d)
                if key in x and x[key].X > 0.5:
                    schedule_rows.append({
                        "date": d,
                        "home": i,
                        "away": j,
                        "home_tz": team_timezone[i],
                        "away_tz": team_timezone[j]
                    })

    schedule_df = pd.DataFrame(schedule_rows).sort_values(["date", "home"])
    print(schedule_df.head(20))

else:
    print("\n!!! No feasible schedule found under all constraints.")
    print("Gurobi status code:", m.status)
    
    if m.status == GRB.INFEASIBLE:
        print("Calculating IIS.")
        m.computeIIS()
        m.write("model.ilp")
        print("IIS computed:")
        for c in m.getConstrs():
            if c.IISConstr:
                print(f"  {c.ConstrName}")


===== Found a feasible schedule (respecting timezone constraint) =====

          date                   home                   away  home_tz  away_tz
0   2025-11-01         Boston Celtics           Phoenix Suns        0       -2
1   2025-11-01          Brooklyn Nets          Atlanta Hawks        0        0
2   2025-11-01          Chicago Bulls        Houston Rockets       -1       -1
3   2025-11-01             Miami Heat  Golden State Warriors        0       -3
4   2025-11-01        Milwaukee Bucks       Dallas Mavericks       -1       -1
5   2025-11-01        New York Knicks     Los Angeles Lakers        0       -3
6   2025-11-01     Philadelphia 76ers    Cleveland Cavaliers        0        0
7   2025-11-01        Toronto Raptors         Denver Nuggets        0       -2
8   2025-11-03          Atlanta Hawks        Toronto Raptors        0        0
9   2025-11-03          Chicago Bulls        Milwaukee Bucks       -1       -1
10  2025-11-03    Cleveland Cavaliers             Miami He

In [ ]:
team = "Golden State Warriors"

for d in dates:
    for i in teams:
        for j in teams:
            if i == j:
                continue
            key = (i, j, d)
            if key in x and x[key].X > 0.5:
                if i == team or j == team:
                    role = "home" if i == team else "away"
                    opp  = j if i == team else i
                    tz_here  = team_timezone[i]
                    rows.append({
                        "date": d,
                        "role": role,
                        "opponent": opp,
                        "venue_tz": tz_here,
                        "team_tz": tz_team
                    })

warriors_df = pd.DataFrame(rows).sort_values("date")
print(warriors_df)

          date  role             opponent  venue_tz  team_tz
0   2025-11-01  away           Miami Heat         0       -3
1   2025-11-03  home       Boston Celtics        -3       -3
2   2025-11-05  home  Cleveland Cavaliers        -3       -3
3   2025-11-07  away       Boston Celtics         0       -3
4   2025-11-11  home      Milwaukee Bucks        -3       -3
5   2025-11-13  away        Chicago Bulls        -1       -3
6   2025-11-15  home   Philadelphia 76ers        -3       -3
7   2025-11-17  home      Houston Rockets        -3       -3
8   2025-11-19  away   Los Angeles Lakers        -3       -3
9   2025-11-21  away         Phoenix Suns        -2       -3
10  2025-11-23  away     Dallas Mavericks        -1       -3
11  2025-11-27  home       Denver Nuggets        -3       -3
12  2025-11-28  away        Atlanta Hawks         0       -3
13  2025-11-29  away        Brooklyn Nets         0       -3
14  2025-12-01  away      Toronto Raptors         0       -3
15  2025-12-25  home    